# Part 2: Long-running MCP Tools and HITL

Learning Objectives:
- Debug and monitor MCP tools with Temporal Web UI
- Implement long running MCP tools with Temporal (queries, signals)

### Instructor-Led Demo (Expand for instructor notes or to run on your own).
<!--
Prep:
1. Clone this repository: `https://github.com/Aslan11/temporal-invoice-mcp/`.
2. Ahead of time, edit `claude_desktop_config.json` and add the full path to the directory containing `server.py`.
3. Make sure you have Claude Desktop installed to your desktop as we will use this as our MCP Client.
4. Run `cp claude_desktop_config.json ~/Library/Application\ Support/Claude/` to connect the tools in this repository to the MCP client in Claude Desktop. Restart Claude Desktop if you have not already.

Demo: 
1. Run your temporal server: `temporal server start-dev`. 
2. In Claude Desktop, prompt it to `Process this invoice for me: {
  "invoice_id": "INV-100",
  "customer": "ACME Corp",
  "lines": [
    {"description": "Widget A", "amount": 100, "due_date": "2024-06-30T00:00:00Z"},
    {"description": "Widget B", "amount": 200, "due_date": "2024-07-05T00:00:00Z"}
  ]
}`.
3. Allow Claude to use the `process_invoice` tool.
4. Go on the Web UI and show that the timer has started to represent a long-running Workflow Execution.
5. Quit your Worker with `Ctrl-C`. Refresh your Web UI and show that the Workflow is still running.
6. Restart your Worker with `python worker.py` and show that the process did not restart. The timer does not restart from the beginning.
-->

### Make Sure Your Temporal Web UI is Running

1. You should have the Temporal Server running in your terminal (run `temporal server start-dev` if not).
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI.

In [ ]:
# Run this code to load it into the program
from temporalio import activity

@activity.defn
async def payment_gateway(line: dict) -> bool:
    activity.logger.info("Paying %s", line.get("description"))
    # Simulate payment processing
    activity.logger.info("Payment succeeded")
    return True

### Durable Timers

Durable timers survive crashes and restarts! Also, during these waiting periods, the Workflow instance is not consuming CPU or memory. The Worker only “wakes up” when a Workflow needs to perform something.

Traditional timers:
```
async def wait_for_approval():
    await asyncio.sleep(432000)
    # If process crashes, timer is LOST
```

Durable timers:

`await workflow.sleep(timedelta(days=5))`

Let's take a brief pause from MCP material to look into durable Timers.

<img src="https://i.postimg.cc/FKLtvT5Y/durable-timer.png" width="200">

_Read more about durable Timers in [this chapter](https://temporal.talentlms.com/unit/view/id:3077) of our free Temporal 102 course._

### Durable Timer Demo: `InvoiceWorkflow` with Durable Timer

Here's our InvoiceWorkflow that includes a durable timer. The workflow will:
1. Process each line item in the invoice
2. Wait 20 seconds using a durable timer before completing
3. Return a success message

The key thing to observe: even if we kill the Worker during the 20-second wait, the timer persists and the Workflow completes successfully when we restart the Worker!

In [ ]:
# Durable Timer Demo: Here is an example of our Workflow with our durable Timer.
# Run this codeblock.
from temporalio import workflow
from datetime import timedelta

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class InvoiceWorkflow:
    @workflow.run
    async def run(self, invoice: dict) -> str:
        for line in invoice.get("lines", []):
            await workflow.execute_activity(
                payment_gateway,
                line,
                start_to_close_timeout=timedelta(seconds=30),
            )
        
        await workflow.sleep(timedelta(seconds=20)) # Our durable timer
        return "The durable timer persists!"

In [ ]:
# Durable Timer Demo: Load the Worker code.
# Run this codeblock
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker():
    client = await Client.connect("localhost:7233")

    worker = Worker(
        client,
        task_queue="invoice-task-queue",
        workflows=[InvoiceWorkflow],
        activities=[payment_gateway],
    )
    print("Worker started. Listening for workflows...")
    await worker.run()

In [ ]:
# Durable Timer Demo: Start the Workflow
# Run this codeblock
from temporalio.client import Client
import uuid

client = await Client.connect("localhost:7233", namespace="default")

handle = await client.start_workflow(
    InvoiceWorkflow,
    {
      "invoice_id": "INV-100",
      "customer": "ACME Corp",
      "lines": [
        {"description": "Widget A", "amount": 100, "due_date": "2024-06-30T00:00:00Z"},
        {"description": "Widget B", "amount": 200, "due_date": "2024-07-05T00:00:00Z"}
      ]
    },
    id=f"durable-timer-workflow-{uuid.uuid4()}",
    task_queue="invoice-task-queue",
)

print(f"Started workflow. Workflow ID: {handle.id}, RunID {handle.result_run_id}")

In [ ]:
# Step 1: Open your Web UI 
    # - In your `Ports` tab on the bottom of this screen, find `8233` and click on the globe icon.
# Step 2: Run the Worker 

import asyncio

worker = asyncio.create_task(run_worker())

In [ ]:
# After about 5 seconds, run this to kill the current Worker
x = worker.cancel()

if x:
  print("Worker killed")
else:
  print("Worker was not running. Nothing to kill")

### Watch the Progress in the Web UI

Refresh your Web UI, click on the running Workflow Execution, and watch the progress. What do you observe? Does the Timer complete despite the Worker being kiled?

Let's finish this Workflow Execution by starting the Worker again.

In [ ]:
import asyncio

worker = asyncio.create_task(run_worker())

### Watch the Progress in the Web UI

Now see the Workflow Execution complete successfully! Now you see the power of durable timers - imagine if your application crashes during a 20-second wait: maybe a server restarts, a container gets killed, or your laptop runs out of battery. In a traditional application, that timer would be lost and you'd have to start over. But with Temporal's durable timer, the countdown continues even when no Worker is running.

Now that you've seen Temporal's durable timer in action, let's go back to our workflow!

In [ ]:
# Step 1: Call the durable timer for two seconds
# Step 2: Execute the `payment_gateway` Activity
# Step 3: In your retry policy, your maximum_interval is the the maximum interval between retries. 
# Set your maximum interval to 30 seconds.
# Step 3: Now run the code to load it into the program

from datetime import timedelta
from temporalio import workflow
from temporalio.common import RetryPolicy

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class InvoiceWorkflow:
    @workflow.run
    async def run(self, invoice: dict) -> str:
        # Process each line item
        for line in invoice.get("lines", []):
            # TODO: Call the durable timer for 2 seconds
            try:
                await workflow.execute_activity(
                    # TODO: Call your payment_gateway Activity
                    line,
                    start_to_close_timeout=timedelta(seconds=30),
                    retry_policy=RetryPolicy(
                        initial_interval=timedelta(seconds=1),
                        # TODO: Set your maximum interval to 30 seconds
                        maximum_attempts=3,
                    ),
                )
            except Exception as e:
                workflow.logger.error(f"Failed to process line item {line.get('item_id')}: {e}")
        return "COMPLETED"

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "durable-timer-solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))


In [ ]:
# Kill any previous workers that may still be running
x = worker.cancel()

# Start a new worker
worker = asyncio.create_task(run_worker())

In [ ]:
# Creating your MCP Tool
# Step 1: Call the `start_workflow` method on client.
# Step 2: Run this codeblock 

# Note: MCP servers cannot be run directly in Jupyter notebooks because
# MCP servers need to run as separate processes that communicate with SSE protocol
# Therefore, later on in the workshop, you will add this yourself to the `mcp_servers` directory.

from temporalio.client import Client
from fastmcp import FastMCP
from typing import Dict

import uuid

# Initialize FastMCP server
mcp = FastMCP("invoice")

# Temporal client setup (do this once, then reuse)
temporal_client = None

async def get_temporal_client():
    """Get or create a Temporal client connection."""
    global temporal_client
    if not temporal_client:
        temporal_client = await Client.connect("localhost:7233")
    return temporal_client

@mcp.tool
async def process_invoice(
    invoice_id: str,
    customer: str, 
    item_description: str,
    amount: float,
    due_date: str
) -> Dict[str, str]:
    """
    Process an invoice by starting the InvoiceWorkflow.
    Use this tool whenever the user asks to process or submit an invoice.
    
    Args:
    invoice_id: Invoice ID (e.g., "INV-100")
    customer: Customer name (e.g., "ACME Corp")
    item_description: Description of the item (e.g., "Widget A")
    amount: Amount in dollars (e.g., 100)
    due_date: Due date in YYYY-MM-DD format (e.g., "2024-06-30")
        
    Returns: Dictionary with workflow_id and run_id
    """

    invoice = {
        "invoice_id": invoice_id,
        "customer": customer,
        "lines": [
            {
                "description": item_description,
                "amount": amount,
                "due_date": due_date
            }
        ]
    }

    client = await get_temporal_client()
    handle = await __ ( # TODO Call the `start_workflow` method on client.
        "InvoiceWorkflow",
        invoice,
        id=f"invoice-{uuid.uuid4()}",
        task_queue="invoice-task-queue",
    )
    return {"workflow_id": handle.id, "run_id": handle.result_run_id}

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "process-invoice-solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))


### Configure Your MCP Client
#### Take 10 minutes to do the following steps.

1. Add your new MCP tool, `process_invoice`, to a file called `invoice.py` - in the `mcp_servers` directory. Follow the structure used in `weather.py`.
2. At the bottom of `invoice.py`, make sure you initialize and run the server like you did at the bottom of `weather.py` (lines 37-39), still running on port 5125.
3. Go to the terminal window where you're running `uv run mcp_servers/weather.py`, stop it with `Ctrl+C`, then run `uv run mcp_servers/invoice.py`.
4. Go to the MCP Client interface from the first chapter and click `Load More Tools`. You should see `process_invoice` as an available tool.
5. In the chat interface, try asking something like: `Process invoice INV-100 for ACME Corp, Widget A for $100 due 2024-06-30`.
6. OpenAI will decide to use the `process_invoice` tool, and your MCP Client will call it on the MCP server.
7. Open your Web UI to see your Workflows executing in real-time!

Now, let's add some human in the loop capabilities to our MCP tool.

### Instructor-Led Demo (Expand for instructor notes or to run on your own).
<!--
Prep:
1. Clone this repository: `https://github.com/Aslan11/temporal-invoice-mcp/`.
2. Ahead of time, edit `claude_desktop_config.json` and add the full path to the directory containing `server.py`.
3. Make sure you have Claude Desktop installed to your desktop as we will use this as our MCP Client.
4. Run `cp claude_desktop_config.json ~/Library/Application\ Support/Claude/` to connect the tools in this repository to the MCP client in Claude Desktop. Restart Claude Desktop if you have not already.

Demo: 
1. Run your temporal server: `temporal server start-dev`. 
2. In Claude Desktop, prompt it to `Process this invoice for me: {
  "invoice_id": "INV-100",
  "customer": "ACME Corp",
  "lines": [
    {"description": "Widget A", "amount": 100, "due_date": "2024-06-30T00:00:00Z"},
    {"description": "Widget B", "amount": 200, "due_date": "2024-07-05T00:00:00Z"}
  ]
}`.
3. Allow Claude to use the `process_invoice` tool.
4. Go on the Web UI and show that the timer has started to represent a long-running Workflow Execution.
5. Quit Claude Desktop once again to show that the Timer is durably stored. The Workflow lives in Temporal not the MCP server.
6. Go back on Claude Desktop and ask it, "What is the status of this invoice?"
7. Allow prompt to use the `invoice_status` tool and point out the status.
8. Finally, prompt Claude to "approve this invoice."
9. Allow Claude to use the `approve_invoice` tool.
10. Show that the Workflow Execution has completed successfully.

## Let's add Signals and Queries to our Workflow!

In [ ]:
# Let's add Signals first.
# Step 1: Add a RejectInvoice Signal which updates the Workflow’s approved field to False.
# Step 2: We want to wait five days for an Signal. In `workflow.wait_condition()`, 
# set the second parameter to 5 days
# Step 3: If there is no approval after 5 days, set the `approved` field to be False
# Step 4: Run this code to load the code into the program
from datetime import timedelta
from temporalio import workflow
from temporalio.common import RetryPolicy

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class InvoiceWorkflow:
    def __init__(self) -> None:
        self.approved: bool | None = None

    @workflow.signal
    async def ApproveInvoice(self) -> None:
        self.approved = True

    # TODO: Add a RejectInvoice Signal which updates the Workflow’s approved field to False.

    @workflow.run
    async def run(self, invoice: dict) -> str:
        # Wait for the approval signal
        await workflow.wait_condition(
            lambda: self.approved is not None,
            timeout=timedelta(days=_), # TODO: Set this to 5 days
        )

        # Auto-reject if no approval happened after 5 days
        if self.approved is None:
            self.approved = # TODO: Set approval to be False
            return "REJECTED"

        # Only process if approved
        # Process each line item
        for line in invoice.get("lines", []):
            await workflow.sleep(timedelta(seconds=2))

            # Try to process payment
            try:
                await workflow.execute_activity(
                    payment_gateway,
                    line,
                    start_to_close_timeout=timedelta(seconds=30),
                    retry_policy=RetryPolicy(
                        initial_interval=timedelta(seconds=1),
                        maximum_interval=timedelta(seconds=30),
                        maximum_attempts=3,
                    ),
                )
            except Exception:
                workflow.logger.error(f"Failed to process line item: {line}")

        return "COMPLETED"

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "reject-invoice-signal-solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))


In [ ]:
# Kill any previous workers that may still be running
x = worker.cancel()

# Start a new worker
worker = asyncio.create_task(run_worker())

In [ ]:
# Creating your MCP Tool to approve/reject invoices
# Step 1: Add an MCP tool which sends RejectSignal to the Workflow

# Note: MCP servers cannot be run directly in Jupyter notebooks because
# MCP servers need to run as separate processes that communicate with SSE protocol
# Therefore, later on in the workshop, you will add this yourself to the `mcp_servers` directory.

from temporalio.client import Client
from fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("invoice")

# Temporal client setup (do this once, then reuse)
temporal_client = None

async def get_temporal_client():
    """Get or create a Temporal client connection."""
    global temporal_client
    if not temporal_client:
        temporal_client = await Client.connect("localhost:7233")
    return temporal_client

@mcp.tool
async def approve_invoice(workflow_id: str, run_id: str) -> str:
    """Signal approval for the invoice workflow."""
    client = await get_temporal_client()
    handle = client.get_workflow_handle(workflow_id=workflow_id, run_id=run_id)
    await handle.signal("ApproveInvoice")
    return "APPROVED"

# TODO: Add an MCP tool, reject_invoice, to send the RejectInvoice Signal

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "reject-invoice-tool-solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Configure Your MCP Client

1. Add your new MCP tools, `approve_invoice` and `reject_invoice`, to `invoice.py` in the `mcp_servers` directory.
2. Restart your MCP sever. Do this by going to the terminal window where you're running `uv run mcp_servers/invoice.py`, stop it with `Ctrl+C`, then run it again.
3. Go to the MCP Client interface from the first chapter and click `Load More Tools`. You should see two new tools now: `approve_invoice` and `reject_invoice`.
4. Clear the chat with the `Clear Chat` button on the bottom left. In the chat interface, ask something like: `Process invoice INV-100 for ACME Corp, Widget A for $100 due 2024-06-30`.
5. OpenAI will decide to use the `process_invoice` tool.
6. Refresh the Web UI to see your Workflows executing in real-time! Notice the timer has started to represent a long-running Workflow Execution and wait for a Signal.
7. Now, in your chat interface, tell your MCP Client to approve the invoice.
8. OpenAI will decide to use the `approve_invoice` tool. What Event does this add in your Event History?
9. Go to the Web UI and see that your Workflow Execution has completed successfully.

### Now let's add our `GetInvoiceStatus` Query

In [ ]:
# Step 1: In the __init__ method, create a new instance variable called `self.status` and set it to "INITIALIZING"
# Step 2: Add a `GetInvoiceStatus` Query that returns the value of `self.status`.
# Step 3: At the end of the workflow, set the `status` field to be "PAID"
# Step 4: Run this code to load the code into the program

from datetime import timedelta
from temporalio import workflow
from temporalio.common import RetryPolicy

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class InvoiceWorkflow:
    def __init__(self) -> None:
        self.approved: bool | None = None
        # TODO: Create a new instance variable called `self.status` and set it to "INITIALIZING"

    @workflow.signal
    async def ApproveInvoice(self) -> None:
        self.approved = True

    @workflow.signal
    async def RejectInvoice(self) -> None:
        self.approved = False

    @workflow.query
    # TODO Add a `GetInvoiceStatus` Query that returns the value of `self.status`.

    @workflow.run
    async def run(self, invoice: dict) -> str:
        self.status = "PENDING-APPROVAL"

        # Wait for the approval signal
        await workflow.wait_condition(
            lambda: self.approved is not None,
            timeout=timedelta(days=5),
        )

       # Auto-reject if no approval happened after 5 days
        if self.approved is None:
            self.approved = False
            self.status = "REJECTED"
            return "REJECTED"

        # Only process if approved
        # Process each line item
        self.status = "APPROVED"
        # Process each line item
        for line in invoice.get("lines", []):
            # Simple delay - wait 2 seconds before processing
            await workflow.sleep(timedelta(seconds=2))

            # Try to process payment
            try:
                await workflow.execute_activity(
                    payment_gateway,
                    line,
                    start_to_close_timeout=timedelta(seconds=30),
                    retry_policy=RetryPolicy(
                        initial_interval=timedelta(seconds=1),
                        maximum_interval=timedelta(seconds=30),
                        maximum_attempts=3,
                    ),
                )
            except Exception:
                workflow.logger.error(f"Failed to process line item: {line}")

        # TODO: Set the `status` field to be "PAID"
        return self.status

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "invoice-status-query-solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Create your MCP Tool to query for status
# Step 1: Pass the Query name "GetInvoiceStatus" as a string to the `handle.query()` method

# Note: MCP servers cannot be run directly in Jupyter notebooks because
# MCP servers need to run as separate processes that communicate with a protocol
# Therefore, later on in the workshop, you will add this yourself to the `mcp_servers` directory.

from temporalio.client import Client
from fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("invoice")

# Temporal client setup (do this once, then reuse)
temporal_client = None

async def get_temporal_client():
    """Get or create a Temporal client connection."""
    global temporal_client
    if not temporal_client:
        temporal_client = await Client.connect("localhost:7233")
    return temporal_client

@mcp.tool
async def invoice_status(workflow_id: str, run_id: str) -> str:
    """Return current status of the workflow."""
    client = await get_temporal_client()
    handle = client.get_workflow_handle(workflow_id=workflow_id, run_id=run_id)
    desc = await handle.describe()
    status = await handle.query() # TODO: Pass the Query name "GetInvoiceStatus" as a string to the `handle.query()` method
    return (
        f"Invoice with ID {workflow_id} is currently {status}. "
        f"Workflow status: {desc.status.name}"
    )

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "notebooks" / "02_MCP_Temporal_HITL_Solution" / "query-tool-solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Kill any previous workers that may still be running
x = worker.cancel()

# Start a new worker
worker = asyncio.create_task(run_worker())

### Configure Your MCP Client

1. Add your new MCP tool, `invoice_status`, to `invoice.py` in the `mcp_servers` directory. 
2. Restart your MCP sever. Do this by going to the terminal window where you're running `uv run mcp_servers/invoice.py`, stop it with `Ctrl+C`, then run it again.
3. Go to the MCP Client interface from the first chapter and click `Load More Tools`. You should now see a fourth tool, `invoice_status`, as an available tool.
    - _Troubleshooting: If your tools aren't loading, verify that port 5125 is set to **Public** in the Ports tab. Right-click the port → Port Visibility → Public._
4. Clear the chat with the `Clear Chat` button on the bottom left. In the chat interface, ask something like: `Process invoice INV-100 for ACME Corp, Widget A for $100 due 2024-06-30`.
5. OpenAI will decide to use the `process_invoice` tool.
6. Ask what the status is of the invoice.
7. OpenAI will use the `invoice_status` tool and point out the status. What Event does this add in your Event History?

In [ ]:
# Kill any worker to prepare for the exercise.
x = worker.cancel()

if x:
  print("Worker killed")
else:
  print("Worker was not running. Nothing to kill")

---
## Exercise 2 - Dealing with Long-Running Workflows and HITL

* In this exercise, you will:
  * Implement a Temporal Signal to send external input to your running Workflow Execution
  * Build a Temporal Query to fetch the latest conversion amount from yourWworkflow state
  * Pause Workflow progress so it waits on Signals or timeouts
  * Add durability to long-running workflows by exposing them through MCP tools
* Go to the **exercises** Directory and open the `02_Long_Running_Workflows_HITL` folder. Then, open the **practice** directory.
* Open the `README.md` and follow the instructions
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers

# Thank you for attending this workshop. Feedback?

Please leave your feedback for this workshop [here](https://docs.google.com/forms/d/e/1FAIpQLSfkHMev6KCNGFHpVNydyjgAh2ALeHNVYv9TaSrAoBsT0KmNHQ/viewform?usp=header)!

### What's Next?

This workshop introduced you to the **importance of durable MCP tools**, long-running MCP tools, and adding human in the loop in your MCP tools. Further your learning with these resources:

### Resources

- [Temporal Docs](https://docs.temporal.io/)
- [Free Temporal Courses](https://learn.temporal.io/courses/)
- [Invoice Demo GitHub Repository](https://github.com/Aslan11/temporal-invoice-mcp/tree/main)
- A More Advanced [MCP+Temporal Webinar](https://www.youtube.com/watch?v=6gVeAiyyt6c)
- Join our [Community Slack](https://t.mp/slack)